In [1]:
import pandas as pd
import numpy as np
from  matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split as train_test_split

from sklearn.preprocessing import MinMaxScaler
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
data_df = pd.read_csv('../data/dataset.csv',index_col=0)

C:\Users\Aditya\Anaconda3\envs\Keras\lib\site-packages\numpy\lib\arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
data_df.head()

,number_of_prev_edits,namespace,is_reverter,target
0,1,0,True,1
1,0,0,True,1
2,1,3,True,1
3,1,0,True,1
4,1,0,False,1


In [4]:
# Convert booleans to ints/reals
data_df = data_df.astype({'is_reverter': float})

Change the data type for float

In [5]:
Y = data_df['target']
data_df.drop('target',axis=1,inplace=True)

Convert namespace to one-hot-encoded vector

In [6]:
namespace = data_df['namespace'].values
encoded = to_categorical(np.array(namespace))
one_hot = pd.DataFrame(encoded,columns=['namespace_'+str(i) for i in range(6)])
data_df = pd.concat([data_df,one_hot], axis=1)
data_df.drop('namespace',axis=1,inplace=True)
data_df.head()

,number_of_prev_edits,is_reverter,namespace_0,namespace_1,namespace_2,namespace_3,namespace_4,namespace_5
0,1,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0


We split the data into train, val and test

In [7]:
X_train, X_test, y_train, y_test = train_test_split(data_df.to_numpy(), Y, test_size=0.2, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

The values are in the range of [0,1] for all columns except for the 'number of prev edits'. We need to scale this feature column to [0,1]

In [8]:
print(X_train.shape)
scaler = MinMaxScaler().fit(X_train[:,0].reshape(-1, 1))

x_tr_col = scaler.transform(X_train[:,0].reshape(-1, 1))
x_te_col = scaler.transform(X_test[:,0].reshape(-1, 1))
x_va_col = scaler.transform(X_val[:,0].reshape(-1, 1))
print(x_tr_col.shape)

X_train = np.delete(X_train,0,1)
X_test = np.delete(X_test,0,1)
X_val = np.delete(X_val,0,1)
print(X_train.shape)

X_train = np.concatenate([x_tr_col,X_train],axis=1)
X_test = np.concatenate([x_te_col,X_test],axis=1)
X_val = np.concatenate([x_va_col,X_val],axis=1)

print(X_train.shape)

(1273126, 8)
(1273126, 1)
(1273126, 7)
(1273126, 8)


## Random Forest Classification

In [9]:
from sklearn.ensemble import RandomForestClassifier

In [10]:
clf_rf = RandomForestClassifier(n_estimators=500, max_depth=3,random_state=0)
clf_rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=3, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [11]:
print(clf_rf.feature_importances_)

[5.23157485e-01 3.52139809e-01 1.37215765e-02 8.83266468e-02
 6.64036910e-03 8.48500262e-04 1.49382546e-02 2.27358630e-04]


In [12]:
clf_rf.score(X_val, y_val, sample_weight=None)

0.6453082486599934

In [13]:
clf_rf.score(X_test, y_test, sample_weight=None)

0.6460517981560983

## Gradient Boosted Classifiers

In [14]:
from sklearn.ensemble import GradientBoostingClassifier

In [16]:
clf_xgb = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
clf_xgb.fit(X_train,y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=1.0, loss='deviance', max_depth=1,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=0, subsample=1.0, verbose=0,
              warm_start=False)

In [17]:
clf_xgb.score(X_val, y_val, sample_weight=None)

0.645232843830314

In [18]:
clf_xgb.score(X_test, y_test, sample_weight=None)

0.6459361772719504